In [ ]:
from pytrends.request import TrendReq
from __future__ import print_function, division

import re
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import datetime

from easymoney.money import EasyPeasy

import pickle

from sklearn.model_selection import train_test_split
#import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')
%matplotlib inline

In [3]:

df = pd.read_csv('fai_scrape.csv')


In [10]:
pkl_file = open('country_numbers.pkl', 'rb')

country_number = pickle.load(pkl_file)

pkl_file.close()
del country_number['']
country_number['0'] = 'Unknown'

In [11]:
ep = EasyPeasy()

In [14]:
#process the data
df = pd.read_csv('fai_scrape.csv')

df = df[df.price != 'Unsold']

df['artist_country'] = df.artist_country.apply(str)
df['artist_country_name'] = df.artist_country.apply(lambda x: country_number[x])
df['price'] = (df.price
                   .apply(lambda x: x[:-3])
                   .apply(lambda x: x.split(','))
                   .apply(lambda x: ''.join(x))
                   .apply(int))

df['height'] = (df['size']
                 .apply(lambda x: re.findall('\d+\.?\d*', x))
                 .apply(lambda x: x[0]))

df['width'] = (df['size']
                .apply(lambda x: re.findall('\d+\.?\d*', x))
                .apply(lambda x: x[1]))
df['area'] = df.height.apply(float) * df.width.apply(float)
df = df[df.area > 0]
df['log_area'] = df['area'].apply(np.log)
df['trans_date'] = pd.to_datetime(df.trans_date, format='%m-%d-%Y')

df['trans_date_ord'] = df['trans_date'].apply(lambda x: x.toordinal())

df['price_norm'] = df.apply(lambda row: ep.normalize(row.price, region='USA', from_year=row.trans_date.year, to_year="latest"), axis= 1)


df['log_price'] = df['price_norm'].apply(np.log)
df = df.dropna()


Add count data

df['prev_count'] = df.sort_values('trans_date').groupby('artist_name').cumcount()
df['log_prev_count'] = df.prev_count.apply(lambda x: np.log(x+1))

In [ ]:
def add_cat_prev_counts(df, col):
    df[col + '_prev_count'] = df.sort_values('trans_date').groupby(col).cumcount()
    df[col + '_lpc'] = df[col + '_prev_count'].apply(lambda x: np.log(x+1))

In [ ]:
add_cat_prev_counts(df, 'wiki_artist_name')


Add wiki data

In [23]:
df_wiki = pd.read_csv('wiki_scrape.csv')

df_wiki['wiki_html_len'] = df_wiki.wiki_content.apply(len)
df_wiki['log_wiki_html'] = df_wiki.wiki_html_len.apply(lambda:x np.log(x+1))
#df_wiki['log_pagerank'] = df_wiki.pagerank.apply(lambda:x np.log(x+1))

In [18]:
movements = ['Gothic', 'Renaissance', 'Mannerism', 'Baroque',
             'Rococo', 'Neoclassical', 'Romanticism', 'Academic',
             'Impressionism', 'Pointillism', 'Symbolism',
             'Art Nouveau', 'Tonalism', 'Expressionism',
             'Cubism', 'Futurism', 'Dada', 'Surrealism']

def get_movements(content):
    out = []
    for move in movements:
        if move in content:
            out.append(move)
    return out

df_wiki['movements'] = df_wiki.wiki_content.apply(get_movements)

In [26]:
merg_cols = ['wiki_artist_name', 'wiki_html_len', 'pagerank', 'log_pagerank']
#merg_cols = ['wiki_artist_name', 'wiki_html_len', 'log_wiki_html']

df_wiki_merge = df_wiki.loc[:,merg_cols]
df = pd.merge(df, df_wiki_merge, on='wiki_artist_name')

In [ ]:
#df.to_csv('wiki_art_clean.csv')

,Unnamed: 0,artist_born,artist_country,artist_died,fai_artist_name,medium,price,size,title,trans_date,...,artist_country_name,height,width,area,log_area,trans_date_ord,price_norm,log_price,wiki_html_len,log_wiki_html
0,0,1923.0,17,1985.0,Herb Aach,Ink,100,11.50 x 8.75 in,"Floral Study, 1981",2008-08-13,...,United States of America,11.50,8.75,100.6250,4.611401,733267,95.22,4.556190,6043,8.706656
1,2,1552.0,13,1616.0,Hans Von Aachen (school),Oil,624,8.03 x 5.31 in,The Deposition.,2015-03-23,...,Germany,8.03,5.31,42.6393,3.752776,735680,539.74,6.291088,8744,9.076123
2,4,1552.0,13,1616.0,Hans Von Aachen (school),Oil,12596,12.24 x 16.38 in,The Last Supper,2013-12-04,...,Germany,12.24,16.38,200.4912,5.300770,735206,11084.99,9.313347,8744,9.076123
3,5,1552.0,13,1616.0,Hans Von Aachen (school),Oil,1085,15.50 x 11.50 in,The Last Supper,2013-09-26,...,Germany,15.50,11.50,178.2500,5.183187,735137,954.85,6.861554,8744,9.076123
4,6,1552.0,13,1616.0,Hans Von Aachen (school),Oil,341,13.58 x 11.02 in,Portrait Study Of A Gentleman,2013-08-20,...,Germany,13.58,11.02,149.6516,5.008310,735100,300.09,5.704082,8744,9.076123
5,7,1552.0,13,1616.0,Hans Von Aachen (school),Oil,17441,12.99 x 18.11 in,The Crucifixion With The Two Thieves,2013-07-05,...,Germany,12.99,18.11,235.2489,5.460644,735054,15348.78,9.638791,8744,9.076123
6,9,1552.0,13,1616.0,Hans Von Aachen (school),Oil,11511,19.09 x 13.94 in,Portrait Of A Young Woman,2013-06-11,...,Germany,19.09,13.94,266.1146,5.583927,735030,10130.14,9.223270,8744,9.076123
7,10,1552.0,13,1616.0,Hans Von Aachen (school),Unknown,263,8.74 x 10.08 in,Hispania,2013-04-25,...,Germany,8.74,10.08,88.0992,4.478463,734983,231.45,5.444364,8744,9.076123
8,12,1552.0,13,1616.0,Hans Von Aachen (school),Oil,4275,8.19 x 6.10 in,A Family Altar With The Holy Family And Saint ...,2012-12-13,...,Germany,8.19,6.10,49.9590,3.911203,734850,3817.28,8.247293,8744,9.076123
9,13,1552.0,13,1616.0,Hans Von Aachen (school),Oil,3289,1.38 x 10.24 in,The Adoration Of The Shepherds,2012-12-13,...,Germany,1.38,10.24,14.1312,2.648385,734850,2936.85,7.985093,8744,9.076123


# Not used


Add name prob

In [ ]:
df2 = pd.read_csv('Name_Probs.csv', encoding='utf-8')
df = pd.merge(df, df2[['Og_artist_name','name_prob']], left_on='artist_name', right_on='Og_artist_name')
df = df.drop('Og_artist_name', 1)
df = df[df.name_prob>-70]
print(len(df))

Add first name frequency

In [ ]:
df3 = pd.read_csv('name_freq.csv')
df3 = df3.drop('Unnamed: 0', 1)

In [ ]:
df['first_name'] = df.artist_name.apply(lambda x: x.split()[0])
df = pd.merge(df, df3, left_on='first_name', right_on='name', how='outer')
df.loc[df.first_name_freq.isnull(), 'first_name_freq'] = 1
df = df.drop('name', 1)
df = df.dropna()
df['log_fnf'] = df.first_name_freq.apply(np.log)

len(df)

In [ ]:
df.head()